<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<p>معرفی <strong><code>Task</code></strong> در .NET دلایل متعددی دارد که همگی به هدف <strong>سادگی در برنامه‌نویسی همزمانی (Concurrency)</strong> و <strong>موازی‌سازی (Parallelism)</strong>، <strong>بهینه‌سازی منابع</strong>، و <strong>ارتقاء کارایی</strong> مربوط می‌شود. استفاده از <strong><code>Thread</code></strong> مستقیم چالش‌های زیادی داشت که <strong><code>Task</code></strong> به حل آن‌ها کمک کرده است. اجازه بده این موضوع را دقیق‌تر بررسی کنیم.</p>
<h3><strong>1. پیچیدگی در مدیریت مستقیم Threadها</strong></h3>
<ul><li>وقتی از <strong><code>Thread</code></strong> مستقیم استفاده می‌کنید، شما مسئول مدیریت کامل چرخه عمر آن هستید:<ul><li>ایجاد (<code>Thread.Start()</code>).</li><li>هماهنگی بین Threadها (Synchronization).</li><li>اتمام (<code>Thread.Join()</code>).</li></ul></li><li>این مدیریت می‌تواند پیچیده، پرهزینه و مستعد خطا (مانند Race Condition یا Deadlock) باشد.</li></ul>
<h4><strong>چگونه <code>Task</code> این مشکل را حل کرد؟</strong></h4>
<ul><li><strong><code>Task</code></strong> به عنوان بخشی از <strong>Task Parallel Library (TPL)</strong> به شما اجازه می‌دهد که روی <strong>کار (Task)</strong> تمرکز کنید، نه روی نحوه اجرای آن.</li><li>مدیریت Threadها، زمان‌بندی، و اجرای همزمان به طور خودکار توسط TPL انجام می‌شود.</li></ul>
<hr>
<h3><strong>2. بهینه‌سازی استفاده از منابع</strong></h3>
<ul><li>Threadهای ایجادشده دستی (Manual Threads):<ul><li>مصرف منابع بالا (هر Thread به طور پیش‌فرض 1 مگابایت حافظه Stack نیاز دارد).</li><li>ایجاد و تخریب Thread هزینه‌بر است.</li></ul></li><li>Thread Pool:<ul><li>برای کاهش هزینه، Thread Pool استفاده می‌شود، اما مدیریت آن پیچیدگی خاصی دارد.</li></ul></li></ul>
<h4><strong>چگونه <code>Task</code> منابع را بهینه می‌کند؟</strong></h4>
<ul><li><strong><code>Task</code></strong> به طور پیش‌فرض از <strong>Thread Pool</strong> استفاده می‌کند و بهینه‌ترین راهکار را برای تخصیص و مدیریت Threadها ارائه می‌دهد.</li><li>وقتی یک <strong>Task</strong> تعریف می‌کنید، نیازی نیست نگران این باشید که چه تعداد Thread یا چگونه اجرا می‌شوند. TPL به صورت پویا این موارد را مدیریت می‌کند.</li></ul>
<hr>
<h3><strong>3. پشتیبانی از برنامه‌نویسی سطح بالاتر</strong></h3>
<ul><li><strong><code>Thread</code></strong> یک ابزار سطح پایین است و برای کارهای ساده نیاز به کدنویسی زیاد و مدیریت دستی دارد.</li><li><strong><code>Task</code></strong> امکان استفاده از امکانات سطح بالاتر را فراهم می‌کند:<ul><li><strong>زمان‌بندی آسان Taskها</strong>.</li><li><strong>اجرای موازی Taskها</strong>.</li><li><strong>بازگشت نتایج (Return Values)</strong>.</li></ul></li></ul>
</div>

In [ ]:
Thread thread = new Thread(() =>
{
    Console.WriteLine("Running in a thread");
});
thread.Start();
thread.Join();

In [ ]:
Task task = Task.Run(() =>
{
    Console.WriteLine("Running in a task");
});
task.Wait(); // منتظر اتمام Task

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h3><strong>4. قابلیت بازگشت مقدار (Return Value)</strong></h3>
<ul><li>با <strong><code>Thread</code></strong>، امکان بازگرداندن مقدار مستقیم وجود ندارد و باید از مکانیزم‌هایی مثل <strong>Shared Variables</strong> یا <strong>Callback</strong> استفاده کنید.</li><li>با <strong><code>Task</code></strong>، می‌توانید مقدار را به راحتی بازگردانید:</li></ul>
</div>

In [ ]:
Task<int> task = Task.Run(() =>
{
    return 42; // بازگرداندن مقدار
});
Console.WriteLine(task.Result); // دسترسی به نتیجه

Task.Run(() =>
{
    Task.Delay(1000).Wait(); 
    Console.WriteLine("Task complete");
}).Wait();
//what happen for main thread and task thread?

42
Task complete


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h3><strong>6. مدیریت آسان‌تر خطاها</strong></h3>
<ul><li>در برنامه‌نویسی با <strong><code>Thread</code></strong>، مدیریت خطاها پیچیده است. اگر یک Thread دچار Exception شود، مدیریت آن به عهده برنامه‌نویس است.</li><li>در <strong><code>Task</code></strong>، خطاها به راحتی از طریق <strong>Task.Exception</strong> یا <strong><code>await</code></strong> مدیریت می‌شوند.</li></ul>
</div>

In [ ]:
try
{
    Task task = Task.Run(() => throw new InvalidOperationException());
    task.Wait();
}
catch (AggregateException ex)
{
    Console.WriteLine($"Caught exception: {ex.InnerException.Message}");
}

- represents a concurrent operation that might or `might not` be backed by a thread.
- They can use the `thread pool` to lessen startup latency
- when recive to `await`, thread come back to pool for other operations

Tasks use `pooled threads` by default, which are `background`
threads. This means that when the `main thread` ends, so do
any tasks that you create.

In [ ]:
Task.Run (() => Console.WriteLine ("Foo"));
Console.ReadLine();

<div dir="rtl">

Taskهای داغ: این Taskها بلافاصله پس از ایجاد، فعال و آماده اجرا هستند. نیازی به صدا زدن هیچ متد اضافی برای شروع کار آن‌ها نیست. Task.Run معمولاً برای ایجاد این نوع از Taskها استفاده می‌شود.
</div>

<div dir="rtl">
Taskهای سرد: در مقابل، Taskهای "سرد" با استفاده از کانستراکتور Task ایجاد می‌شوند و تا زمانی که صراحتاً با فراخوانی متد Start شروع به کار نکنند، فعال نمی‌شوند. این رویکرد به ندرت در عمل استفاده می‌شود زیرا مدیریت آن دشوارتر است و برنامه‌نویسان معمولاً ترجیح می‌دهند کار با Taskهایی که بلافاصله فعال می‌شوند و نیازی به مرحله اضافی برای شروع ندارند.
</div>

Calling `Wait` on a task blocks current thread until it completes and is the equivalent of calling `Join` on a thread

### Long-running tasks

`tasks` ideal for short-running compute-bound work.

<div dir="rtl">
برای عملیات‌های طولانی‌مدت یا مسدودکننده، استفاده مستقیم از نخ‌های استخر نخ می‌تواند منجر به کاهش عملکرد شود چرا که این نخ‌ها برای کارهای کوتاه‌مدت طراحی شده‌اند و می‌توانند با اشغال شدن در طولانی‌مدت، منابع را محدود کنند.
<br>
برای مدیریت این نوع عملیات‌ها، .NET امکانی فراهم می‌کند تا Taskهایی با گزینه TaskCreationOptions.LongRunning ایجاد کنید. این گزینه به CLR اطلاع می‌دهد که Task مورد نظر احتمالا زمان زیادی برای اجرا نیاز دارد. در نتیجه، به جای استفاده از نخ‌های استخر نخ، CLR ممکن است تصمیم بگیرد یک نخ جدید برای اجرای این Task اختصاص دهد تا اطمینان حاصل شود که عملکرد استخر نخ تحت تأثیر قرار نمی‌گیرد.
</div>

In [ ]:
Task task = Task.Factory.StartNew (() => {/*LongRunning task*/},
TaskCreationOptions.LongRunning);

## Returning values

In [ ]:
Task<int> task = Task.Run (() =>
{ 
    Task.Delay(1000).Wait();

    Console.WriteLine ("Foo"); 
    
    return 3; 
});

int result = task.Result; // Blocks if not already finished
Console.WriteLine (result); // 3

In [ ]:
Task<int> primeNumberTask = Task.Run (() =>
    Enumerable.Range (2, 3000000)
        .Count (n =>
            Enumerable.Range (2, (int)Math.Sqrt(n)-1).All (i => n % i > 0)));
Console.WriteLine ("Task running...");
Console.WriteLine ("The answer is " + primeNumberTask.Result);

Task running...
The answer is 216816


## Exceptions

In [ ]:
Task task = Task.Run (() => { throw null; });
try
{
    task.Wait();//or task.Result if task has return value
}
catch (AggregateException aex)
{
    if (aex.InnerException is NullReferenceException)
        Console.WriteLine ("Null!");
    else
        throw;
}

if(task.IsFaulted) //check for Fault
    Console.WriteLine("Fault");

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h4>1. <strong>اجرای کد داخل <code>Task.Run</code></strong></h4>

<ul>
<li>وقتی <code>Task.Run</code> فراخوانی می‌شود، وظیفه (Task) روی یک نخ جداگانه در <strong>Thread Pool</strong> شروع به اجرا می‌کند.</li>
<li>
در این مثال: <br/>
<ul>
<li>وظیفه بلافاصله شروع به کار می‌کند.</li>
<li>اما چون وظیفه روی یک نخ جداگانه اجرا می‌شود، استثنای پرتاب شده (در اینجا <code>throw null;</code>) <strong>مستقیماً رشته اصلی را تحت تأثیر قرار نمی‌دهد</strong>.</li>
</ul>
</li>
</ul>

<h4>2. <strong>مدیریت استثناها در <code>Task</code></strong></h4>
<ul><li><p>اگر داخل یک <code>Task</code> استثنا پرتاب شود، این استثنا به صورت داخلی توسط خود <code>Task</code> ذخیره می‌شود و تا زمانی که به آن دسترسی پیدا کنید (مثلاً با <code>Wait</code> یا <code>Result</code>)، در برنامه منتشر نمی‌شود.</p></li><li><p>در این مثال:</p><ul><li>با فراخوانی <code>Wait</code> یا دسترسی به <code>Result</code>، <strong>استثنای ذخیره‌شده در <code>Task</code> منتشر می‌شود</strong>.</li><li>این انتشار استثنا در قالب یک <strong><code>AggregateException</code></strong> انجام می‌شود که تمام استثناهای پرتاب شده در <code>Task</code> را جمع‌آوری کرده است.</li></ul></li></ul>

<h4>3. <strong>نقش <code>AggregateException</code></strong></h4>
<ul><li>استثناهای داخل یک <code>Task</code> همیشه در یک <strong><code>AggregateException</code></strong> بسته‌بندی می‌شوند.</li><li>اگر کد شما <code>throw null;</code> اجرا کند، این استثنا به صورت یک <strong><code>NullReferenceException</code></strong> در <code>AggregateException.InnerException</code> ذخیره می‌شود.</li></ul>

</div>

## Continuations

A `continuation` says to a task, “When you’ve finished, continue by doing something else.”

In [ ]:
Task task = Task.Run(() =>
{
    Console.WriteLine("Task 1 running");
    Task.Delay(1000).Wait(); // شبیه‌سازی تأخیر
    Console.WriteLine("Task 1 complete");
});

task.ContinueWith(t =>
{
    Console.WriteLine("Continuation running after Task 1");
})
//.Wait();


Task 1 running
Task 1 complete
Continuation running after Task 1


In [ ]:
// with return value
Task<int> task = Task.Run(() =>
{
    Console.WriteLine("Calculating...");
    return 42; // بازگرداندن نتیجه
});

task.ContinueWith(t =>
{
    Console.WriteLine($"Result from Task: {t.Result}");
}).Wait();


In [ ]:
//with Exception
Task task = Task.Run(() =>
{
    throw new InvalidOperationException("Something went wrong!");
});

task.ContinueWith(t =>
{
    if (t.IsFaulted)
    {
        Console.WriteLine($"Error: {t.Exception?.GetBaseException().Message}");
    }
}).Wait();


In [6]:
// multiple
Task.Run(() =>
{
    Console.WriteLine("Task 1 running");
    return 42;
})
.ContinueWith(t =>
{
    Console.WriteLine($"Task 1 result: {t.Result}");
    Console.WriteLine("Task 2 running");
    return t.Result + 10;
})
.ContinueWith(t =>
{
    Console.WriteLine($"Task 2 result: {t.Result}");
    Console.WriteLine("Task 3 running");
}).Wait();


Task 1 running
Task 1 result: 42
Task 2 running
Task 2 result: 52
Task 3 running


In [5]:
Task<int> primeNumberTask = Task.Run (() =>
    Enumerable.Range (2, 3000000).Count (n =>
        Enumerable.Range (2, (int)Math.Sqrt(n)-1).All (i => n % i > 0)));
var awaiter = primeNumberTask.GetAwaiter();
awaiter.OnCompleted (() =>
{
    int result = awaiter.GetResult();
    Console.WriteLine (result); // Writes result
});

In [ ]:
primeNumberTask.ContinueWith (antecedent =>
{
int result = antecedent.Result;
Console.WriteLine (result); // Writes 123
});

## TaskCompletionSource

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">

TaskCompletionSource<T> یک ابزار قدرتمند در .NET است که به توسعه‌دهندگان امکان می‌دهد به صورت دستی Task<T>ها را ایجاد و کنترل کنند. این کلاس به شما اجازه می‌دهد تا یک Task را بدون نیاز به ایجاد یک thread جدید مدیریت کنید، که بسیار مفید است برای عملیات‌های غیرهمزمان که به صورت طبیعی بر اساس callbackها هستند، مانند درخواست‌های I/O.

<p>
شما می‌توانید دقیقا تعیین کنید که چه زمانی یک Task به حالت کامل شده (Completed)، لغو شده (Canceled)، یا با خطا (Faulted) می‌رود.
<br>
سیار مناسب برای برنامه‌نویسی رویداد محور و سناریوهایی که عملیات غیرهمزمان بر اساس وقایع خارجی اتفاق می‌افتند.
</p>
</div>

In [ ]:
var tcs = new TaskCompletionSource<int>(); // نوع نتیجه Task مشخص می‌شود.
Task<int> task = tcs.Task;                // Task مربوط به این TaskCompletionSource

// تکمیل Task
tcs.SetResult(42); // نتیجه را تنظیم می‌کنید
Console.WriteLine(task.Result); // خروجی: 42

In [ ]:
var tcs = new TaskCompletionSource<int>();
Task<int> task = tcs.Task;

// شکست Task
tcs.SetException(new InvalidOperationException("Something went wrong!"));

// مدیریت استثنا
try
{
    Console.WriteLine(task.Result); // اینجا استثنا پرتاب می‌شود.
}
catch (AggregateException ex)
{
    Console.WriteLine(ex.InnerException?.Message); // خروجی: Something went wrong!
}

In [ ]:
var tcs = new TaskCompletionSource<int>();
Task<int> task = tcs.Task;

// لغو Task
tcs.SetCanceled();

// بررسی لغو Task
Console.WriteLine(task.IsCanceled); // خروجی: True

In [ ]:
using System.Threading;

public static class StockChecker
{
    // متد غیرهمزمان که از یک callback استفاده می‌کند
    public static void CheckStockAsync(int productId, Action<bool> callback)
    {
        // شبیه‌سازی عملیات ناهمزمان
        Task.Run(() =>
        {
            // شبیه‌سازی تأخیر برای بررسی موجودی در انبار
            Thread.Sleep(2000);

            // فرض کنید ما بررسی کردیم و نتیجه پیدا شد
            bool inStock = (productId % 2 == 0); // به صورت فرضی: محصولات با ID زوج موجود هستند

            // فراخوانی callback با نتیجه
            callback(inStock);
        });
    }
}


public Task<bool> IsProductInStockAsync(int productId)
{
    var tcs = new TaskCompletionSource<bool>();

    // فرض کنید CheckStock یک متد غیرهمزمان است که وقتی موجودی چک شد، یک event را فایر می‌کند
    StockChecker.CheckStockAsync(productId, (inStock) => 
    {
        if(inStock)
        {
            tcs.SetResult(true); // محصول در انبار موجود است
        }
        else
        {
            tcs.SetResult(false); // محصول در انبار موجود نیست
        }
    });

    return tcs.Task;
}

Console.WriteLine("Checking stock...");

bool isProductInStockAsync = await IsProductInStockAsync(123);

Console.WriteLine(isProductInStockAsync ? "Product is in stock!" : "Product is out of stock!");


Checking stock...
Product is out of stock!


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<p> چگونه می‌توان از <strong><code>TaskCompletionSource</code></strong> برای ساختن <code>Task</code>‌هایی استفاده کرد که بدون استفاده از نخ‌ها (<strong>threads</strong>) و به صورت کارآمد عمل می‌کنند. این روش به کمک ابزارهایی مانند <strong><code>Timer</code></strong> امکان‌پذیر است که بر اساس رویدادها کار می‌کنند.</p>

</div>

In [ ]:
Task<int> GetAnswerToLife()
{
    var tcs = new TaskCompletionSource<int>();
    // Create a timer that fires once in 5000 ms:
    var timer = new System.Timers.Timer (5000) { AutoReset = false };
    timer.Elapsed += delegate { timer.Dispose(); tcs.SetResult (42); };
    timer.Start();
    return tcs.Task;
}

In [ ]:
Task Delay (int milliseconds)
{
    var tcs = new TaskCompletionSource<object>();
    var timer = new System.Timers.Timer (milliseconds) { AutoReset = false };
    timer.Elapsed += delegate { timer.Dispose(); tcs.SetResult (null); };
    timer.Start();
    return tcs.Task;
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h3><strong> Synchronous، Concurrency و Asynchronous </strong></h3>
<h3><strong>1. Synchronous (همزمان)</strong></h3>
<ul><li><p><strong>تعریف:</strong></p><ul><li>در برنامه‌نویسی همزمان، هر عملیات باید به‌طور کامل انجام شود تا کنترل به عملیات بعدی منتقل شود.</li><li>به عبارت دیگر، تا زمانی که یک کار تکمیل نشده، هیچ کار دیگری نمی‌تواند شروع شود.</li></ul></li><li><p><strong>ویژگی‌ها:</strong></p><ul><li>اجرای ترتیبی.</li><li><strong>Blocking:</strong> نخ جاری در طول عملیات بلاک می‌شود.</li></ul></li><li><p><strong>مثال:</strong>
تصور کنید در یک رستوران، پیشخدمت فقط می‌تواند یک سفارش را بگیرد، منتظر آماده شدن غذا باشد و سپس سفارش را به مشتری برگرداند. تا زمانی که سفارش اول تمام نشود، نمی‌تواند به مشتری دیگری خدمت کند.</p></li></ul>
</div>

In [ ]:
void SynchronousOperation()
{
    Console.WriteLine("Start");
    Thread.Sleep(3000); // شبیه‌سازی عملیات طولانی
    Console.WriteLine("End");
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h4><strong>توضیح:</strong></h4>
<ul><li>نخ (Thread) جاری در طول <code>Thread.Sleep</code> بلاک می‌شود و هیچ کار دیگری نمی‌تواند انجام دهد.</li></ul>
<hr>
<h3><strong>2. Concurrency (همزمانی)</strong></h3>
<ul><li><p><strong>تعریف:</strong></p><ul><li>همزمانی به معنای توانایی اجرای چند کار به صورت موازی است.</li><li>این کار ممکن است با استفاده از <strong>چند نخ (Threads)</strong> یا مکانیزم‌های دیگر انجام شود.</li></ul></li><li><p><strong>ویژگی‌ها:</strong></p><ul><li><strong>Parallelism ممکن است:</strong> همزمانی می‌تواند منجر به اجرای موازی شود، اما لزوماً همه‌ی عملیات‌ها به صورت دقیقاً همزمان اجرا نمی‌شوند.</li><li>از منابع سیستم به صورت بهینه‌تر استفاده می‌شود.</li></ul></li><li><p><strong>مثال:</strong>
در رستوران، پیشخدمت می‌تواند به چند مشتری به صورت همزمان خدمت کند. او سفارش اول را می‌گیرد، سفارش دوم را می‌گیرد، و سپس به آشپزخانه می‌رود و منتظر آماده شدن هر دو سفارش می‌ماند.</p></li></ul>
<div>

In [ ]:
void ConcurrentOperations()
{
    Task.Run(() => 
    {
        Thread.Sleep(3000); // شبیه‌سازی کار اول
        Console.WriteLine("Task 1 completed");
    });

    Task.Run(() =>
    {
        Thread.Sleep(2000); // شبیه‌سازی کار دوم
        Console.WriteLine("Task 2 completed");
    });

    Console.WriteLine("Both tasks started");
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h4><strong>توضیح:</strong></h4>
<ul><li>دو Task به صورت همزمان شروع می‌شوند و هرکدام روی یک نخ جداگانه اجرا می‌شوند.</li><li>ترتیب اجرای "Task 1 completed" و "Task 2 completed" ممکن است به مدت زمان اجرای هر Task بستگی داشته باشد.</li></ul>
<hr>
<h3><strong>3. Asynchronous (ناهمزمان)</strong></h3>
<ul><li><p><strong>تعریف:</strong></p><ul><li>در برنامه‌نویسی ناهمزمان، عملیات می‌تواند بدون بلاک کردن نخ جاری اجرا شود.</li><li>عملیات طولانی‌مدت (مانند خواندن فایل یا انتظار برای پاسخ شبکه) بدون استفاده از نخ‌های اضافی مدیریت می‌شوند.</li></ul></li><li><p><strong>ویژگی‌ها:</strong></p><ul><li>نخ جاری آزاد است تا به کارهای دیگر بپردازد.</li><li>با <strong>async/await</strong> یا سایر مکانیزم‌های مبتنی بر رویداد پیاده‌سازی می‌شود.</li></ul></li><li><p><strong>مثال:</strong>
در رستوران، پیشخدمت سفارش مشتری را به آشپزخانه می‌دهد و بلافاصله به سراغ مشتری بعدی می‌رود. وقتی غذا آماده شد، آشپزخانه به او اطلاع می‌دهد و او سفارش را تحویل می‌دهد.</p></li></ul>
</div>

In [ ]:
async Task AsynchronousOperation()
{
    Console.WriteLine("Start");
    await Task.Delay(3000); // شبیه‌سازی عملیات طولانی
    Console.WriteLine("End");
}


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn; text-align:justify">
<h4><strong>توضیح:</strong></h4>
<ul><li>متد <code>Task.Delay</code> بدون بلاک کردن نخ اجرا می‌شود.</li><li>نخ جاری می‌تواند در این زمان به کارهای دیگر بپردازد.</li></ul>
</div>